In [1]:
import pandas as pd, numpy as np, os, glob

In [2]:
PeriHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_2693_PeriHD.csv')
IntraHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_1124_IntraHD.csv')
PeriHD_Test = pd.read_csv('/Users/monoaki/vs_code/亞東/encrypt/baseline_01_sample_500-test_periHD.csv')
IntraHD_Test = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/IntraHD_ExternalData_05-17.csv')

/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_70352/1004895937.py:1: DtypeWarning: Columns (6,7,8,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  PeriHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_2693_PeriHD.csv')
/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_70352/1004895937.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  IntraHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_1124_IntraHD.csv')


In [3]:
IntraHD_Train['處置其他+症狀處置'] = IntraHD_Train['處置其他']+'||'+IntraHD_Train['症狀處置']
print(IntraHD_Train.shape)
IntraHD_Train = IntraHD_Train.drop_duplicates(subset=['處置其他+症狀處置'])
print(IntraHD_Train.shape)

(1452843, 25)
(1124, 25)


In [4]:
print(PeriHD_Train.shape)
PeriHD_Train = PeriHD_Train.drop_duplicates(subset=['處置其他結束'])
PeriHD_Train = pd.concat([PeriHD_Train,PeriHD_Train.sample(307, random_state=123)])
PeriHD_Train['透析時間長度'] = pd.to_datetime(PeriHD_Train['透析時間結束'])-pd.to_datetime(PeriHD_Train['透析開始時間'])
PeriHD_Test['透析時間長度'] = pd.to_datetime(PeriHD_Test['透析時間結束'])-pd.to_datetime(PeriHD_Test['透析開始時間'])
print(PeriHD_Train.shape)
print(PeriHD_Test.shape)

(61160, 64)
(2693, 65)
(500, 66)


In [5]:
age_df = pd.read_csv('../../../encrypt/Base2016_20_en.csv')
print(age_df.shape)
age_df['2020_Date'] = pd.to_datetime('2020-06-30 12:00:00')
age_df['生日']= pd.to_datetime(age_df['生日'])
age_df['age'] = (age_df['2020_Date'] - age_df['生日'])/365
age_df['age'] = age_df['age'].astype(str).str.split(' days').str[0]
age_df_PeriHD_train = pd.merge(PeriHD_Train['ID'], age_df, on='ID', how='inner')
print(age_df_PeriHD_train.shape)
PeriHD_Test = PeriHD_Test.rename(columns={'病人ID':"ID"})
age_df_PeriHD_test = pd.merge(PeriHD_Test['ID'], age_df, on='ID', how='inner')
print(age_df_PeriHD_test.shape)

(4017, 3)
(2693, 5)
(500, 5)


In [6]:
PeriHD_Train.columns

Index(['ID', '洗腎紀錄時間', '急診門診住院', '護理交班事項', '透析開始時間', '開始血壓SBP', '開始血壓DBP',
       '開始脈搏', '體重1開始', '體重2標準體重', '體重3應脫水', '體重其他', '體溫', 'FISTULA', 'GRAFT',
       'DLC', 'PERMCATH', 'EXITSITE', 'DLC順暢度', '血流強度', '血管彈性', '瘀青', '感染徵兆',
       '穿刺', '血管通路理學篩檢其他', '跌倒_年齡', '跌倒_跌倒病史', '跌倒_意識狀態', '跌倒_下肢肌力測驗',
       '跌倒_如廁能力', '跌倒_活動狀態', '跌倒_藥物使用', '入院方式', '意識狀態', '呼吸系統', '呼吸系統5其他',
       '心臟血管系統', '心臟血管系統其他', '透析時間結束', '結束血壓SBP', '結束血壓DBP', '結束脈搏', '體重結束',
       '體重實際脫水', '體重機器顯示', '血管廔管', '血管廔管2', '血管廔管3', 'Hollow fiber clot',
       'V-chamber clot', 'A-chamber clot', '處置其他結束', '透析後評估項目1', '透析後評估項目2',
       '透析後評估項目3', '透析後評估項目4', '透析後評估項目5', '透析後評估項目6', 'DIAG585',
       'DIAG585 STATDATE', 'DIAG585 ENDDATE', 'NEW開始血壓SBP', 'NEW開始血壓DBP',
       'INDEX', '透析時間長度'],
      dtype='object')

In [7]:
IntraHD_Train.columns

Index(['Unnamed: 0', 'ID', '洗腎紀錄時間', '急診門診住院', '中記錄時間', '記錄者', '血壓', '脈搏',
       '呼吸', '血液流速', '靜脈壓', 'TMPDP', '超過濾率', '溫度', '傳導濃度', '管路固定', '生理鹽水',
       '肝素使用量', '處置其他', '症狀處置', 'RowNum', 'sbp', 'dbp', 'INDEX', '處置其他+症狀處置'],
      dtype='object')

In [79]:
def per_hours(dataframe):
    time_delta = [4 if time.components.hours>5  else time.components.hours for time in pd.to_timedelta(dataframe.dropna())]
    mean = round(np.mean(time_delta),1)
    return f"{mean} ({round(np.std(time_delta),1)})"
def weight_calc(dataframe):
    dataframe_pos = dataframe[['體重1開始','體重機器顯示','體重結束']]
    dataframe_pos = dataframe_pos.apply(pd.to_numeric, errors='coerce')
    dataframe_pos = dataframe_pos.dropna()
    dataframe_pos = dataframe_pos.reset_index()
    dataframe_pos['洗後體重'] = dataframe_pos['體重1開始']-dataframe_pos['體重機器顯示']
    for i in range(len(dataframe_pos)):
        if dataframe_pos.loc[i,'洗後體重'] <30:
            dataframe_pos.loc[i,'洗後體重'] = dataframe_pos.loc[i,'體重結束']
        if dataframe_pos.loc[i,'洗後體重'] >100:
            dataframe_pos.loc[i,'洗後體重'] = dataframe_pos.loc[i,'體重結束']
    dataframe_pos['乾體重'] = dataframe_pos['體重結束']
    return dataframe_pos

def get_BP(dataframe):
    dataframe = dataframe[['NEW開始血壓SBP','NEW開始血壓DBP','開始脈搏','結束血壓SBP','結束血壓DBP','結束脈搏']]
    dataframe = dataframe.apply(pd.to_numeric, errors='coerce')
    dataframe = dataframe.dropna().astype(float)
    dataframe = dataframe[(dataframe['NEW開始血壓SBP']>=40) & (dataframe['NEW開始血壓SBP']<=240) & (dataframe['結束血壓SBP']>=40) & (dataframe['結束血壓SBP']<=240)]
    dataframe = dataframe[(dataframe['NEW開始血壓DBP']>=30) & (dataframe['NEW開始血壓DBP']<=140) & (dataframe['結束血壓DBP']>=30) & (dataframe['結束血壓DBP']<=140)]
    return dataframe

Table_1_Variables = {"PeriHD Variables":['Participants, n', 'Age, years', 'Men, n(%)', 'Weight, kg'],
                     "Internal Data":[2693, 
                                      f"{round(np.mean(age_df_PeriHD_train['age'].astype(int).values),1)} ({round(age_df_PeriHD_train['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_PeriHD_train['性別']=='男')} ({round(sum(age_df_PeriHD_train['性別']=='男')/len(age_df_PeriHD_train)*100,1)})%",
                                      f"{round(np.mean(PeriHD_Train['體重1開始'].fillna(0).astype(float)),1)} ({round(PeriHD_Train['體重1開始'].astype(float).std(),1)})"
                                      ],
                     "External Data":[500,
                                      f"{round(np.mean(age_df_PeriHD_test['age'].astype(int).values),1)} ({round(age_df_PeriHD_test['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_PeriHD_test['性別']=='男')} ({round(sum(age_df_PeriHD_test['性別']=='男')/len(age_df_PeriHD_test)*100,1)})%",
                                      f"{round(np.mean(PeriHD_Test['體重1開始'].fillna(0).astype(float)),1)} ({round(PeriHD_Test['體重1開始'].astype(float).std(),1)})"
                                      ]}
# 暫時移除Blood flow rate：來源於IntraHD
Table_1_Dialysis_information = {"PeriHD Variables":['Hours per session, hour','Dry body wieght, kg','Pre-dialysis weight, kg','post-dialysis, kg'],
                                "Internal Data":[per_hours(PeriHD_Train['透析時間長度']),
                                                 f"{round(np.mean(weight_calc(PeriHD_Train)['乾體重']),1)} ({round(np.std(weight_calc(PeriHD_Train)['乾體重']),1)})",
                                                 f"{round(np.mean(PeriHD_Train['體重1開始'].fillna(0).astype(float)),1)} ({round(PeriHD_Train['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(PeriHD_Train)['洗後體重']),1)} ({round(np.std(weight_calc(PeriHD_Train)['洗後體重']),1)})",
                                                 ],
                                "External Data":[per_hours(PeriHD_Test['透析時間長度']),
                                                 f"{round(np.mean(weight_calc(PeriHD_Test)['乾體重']),1)} ({round(np.std(weight_calc(PeriHD_Test)['乾體重']),1)})",
                                                 f"{round(np.mean(PeriHD_Test['體重1開始'].fillna(0).astype(float)),1)} ({round(PeriHD_Test['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(PeriHD_Test)['洗後體重']),1)} ({round(np.std(weight_calc(PeriHD_Test)['洗後體重']),1)})",
                                                 ]} 
Table_1_Vascular_access = {"PeriHD Vascular access, n(%)": ['AV-Fistula','AV-Graft','Catheter'],
                           "Internal Data":[],
                           "External Data":[]}
Table_1_Blood_pressures = {"PeriHD Variables":['Start-SBP, mmHg','Start-DBP, mmHg','Start-HR, beat/min','End-SBP, mmHg','End-DBP, mmHg','End-HR, beat/min'],
                           "Internal Data":[f"{round(np.mean(get_BP(PeriHD_Train)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(PeriHD_Train)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Train)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(PeriHD_Train)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Train)['開始脈搏']),1)} ({round(np.std(get_BP(PeriHD_Train)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Train)['結束血壓SBP']),1)} ({round(np.std(get_BP(PeriHD_Train)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Train)['結束血壓DBP']),1)} ({round(np.std(get_BP(PeriHD_Train)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Train)['結束脈搏']),1)} ({round(np.std(get_BP(PeriHD_Train)['結束脈搏']),1)})",
                                            ],
                           "External Data":[f"{round(np.mean(get_BP(PeriHD_Test)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(PeriHD_Test)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Test)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(PeriHD_Test)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Test)['開始脈搏']),1)} ({round(np.std(get_BP(PeriHD_Test)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Test)['結束血壓SBP']),1)} ({round(np.std(get_BP(PeriHD_Test)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Test)['結束血壓DBP']),1)} ({round(np.std(get_BP(PeriHD_Test)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(PeriHD_Test)['結束脈搏']),1)} ({round(np.std(get_BP(PeriHD_Test)['結束脈搏']),1)})",
                                            ]}

In [80]:
pd.concat([pd.DataFrame(Table_1_Variables),pd.DataFrame(Table_1_Dialysis_information), pd.DataFrame(Table_1_Blood_pressures)], axis=0)

,PeriHD Variables,Internal Data,External Data
0,"Participants, n",2693,500
1,"Age, years",67.4 (13.1),64.3 (11.5)
2,"Men, n(%)",1336 (49.6)%,290 (58.0)%
3,"Weight, kg",61.1 (23.7),63.0 (14.2)
0,"Hours per session, hour",3.4 (0.7),3.6 (0.6)
1,"Dry body wieght, kg",59.3 (19.3),61.5 (13.0)
2,"Pre-dialysis weight, kg",61.1 (23.7),63.0 (14.2)
3,"post-dialysis, kg",59.2 (17.2),61.6 (13.0)
0,"Start-SBP, mmHg",139.6 (25.2),144.5 (27.7)
1,"Start-DBP, mmHg",71.8 (14.5),73.8 (14.5)


In [37]:
Table_1_Vascular_access

{'PeriHD Vascular access, n(%)': ['AV-Fistula', 'AV-Graft', 'Catheter'],
 'Internal Data': [],
 'External Data': []}